In [2]:
import torch

from pathlib import Path

from app.model.config import Config
from app.model.time_series_embedding import TimeSeriesEmbedding

data_dir = Path('app/data')
config = Config(data_dir)
time_series_embedding = TimeSeriesEmbedding(config)
input = torch.randn(32, config.seq_len, config.input_dims)
output = time_series_embedding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 5]); output.shape=torch.Size([32, 5, 150])'

In [3]:
import torch
from app.model.config import Config
from app.model.positional_encoding import PositionalEncoding

data_dir = Path('app/data')
config = Config(data_dir)
positional_encoding = PositionalEncoding(config)
input = torch.randn((32, config.seq_len, config.d_model))
output = positional_encoding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 150]); output.shape=torch.Size([32, 5, 150])'

In [4]:
import torch
from app.model.config import Config
from app.model.stock_transformer import StockTransformer


data_dir = Path('app/data')
config = Config(data_dir)
input = torch.randn((config.batch_size, config.seq_len, config.input_dims),
                    dtype=torch.float)
display(f'{input.shape=}')
transformer = StockTransformer(config)
output = transformer(input)
display(f'{output.shape=}')

'input.shape=torch.Size([16, 5, 5])'

'output.shape=torch.Size([16, 1])'

In [4]:
import csv
import yfinance as yf

from pathlib import Path


ticker = 'SPY'
data = yf.Ticker(ticker)
hist = data.history(period='max', interval='1d')

if hist.empty:
    print(f'No data available for {ticker}')
else:
    date_from = hist.index.min().strftime('%Y%m%d')
    date_to = hist.index.max().strftime('%Y%m%d')

    out_dir = Path('app/data') / 'tickers'
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{ticker}_{date_from}_{date_to}.csv"

    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    with out_path.open('w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(columns)
        for row in hist.itertuples():
            writer.writerow([getattr(row, c) for c in columns])

    print(f'Saved {out_path}')

Saved app/data/tickers/SPY_19930129_20251231.csv


In [9]:
import torch
from pathlib import Path

from app.model.checkpointer import Checkpointer

model_path = Path('app/data/models/4000/SPY.pkl')
model, config, step = Checkpointer.load_checkpoint(model_path)
input = torch.tensor([
    [[24.25864879369397, 24.25864879369397,
        24.137958998700466, 24.24140739440918, 1003200, 0.000000],
     [24.258645785056938, 24.413818359374996,
        24.258645785056938, 24.413818359375, 480500, 0.007087],
     [24.396570547102538, 24.482777510166507,
        24.344846369264157, 24.46553611755371, 201300, 0.002116],
     [24.500017336389178, 24.74139681753587,
        24.4827759448787, 24.72415542602539, 529400, 0.010515],
     [24.810362901952534, 24.879328469435375,
        24.534500632021164, 24.827604293823242, 531500, 0.004175]]
])
output = model(input)
display(input, output)
last_close = input[:, -1, -3].item()
predicted_log_return = output[0][0]
predicted_close = last_close * torch.exp(predicted_log_return)
display(last_close, predicted_close.item())

Model loaded from app/data/models/4000/SPY.pkl


tensor([[[2.4259e+01, 2.4259e+01, 2.4138e+01, 2.4241e+01, 1.0032e+06,
          0.0000e+00],
         [2.4259e+01, 2.4414e+01, 2.4259e+01, 2.4414e+01, 4.8050e+05,
          7.0870e-03],
         [2.4397e+01, 2.4483e+01, 2.4345e+01, 2.4466e+01, 2.0130e+05,
          2.1160e-03],
         [2.4500e+01, 2.4741e+01, 2.4483e+01, 2.4724e+01, 5.2940e+05,
          1.0515e-02],
         [2.4810e+01, 2.4879e+01, 2.4535e+01, 2.4828e+01, 5.3150e+05,
          4.1750e-03]]])

tensor([[0.0023]], grad_fn=<AddmmBackward0>)

24.827604293823242

24.8835506439209